# Run the CICI SAFE vSDX Using Chameleon

#### Configure the Envrionment

In [19]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
#export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
#export OS_REGION_NAME='CHI@TACC'
#export OS_REGION_NAME='CHI@UC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter


#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter-Tridentcom
export UNIVERSAL_INFO_FILE=${WORKING_DIR}/universal_info
export SDX_INFO_FILE=${WORKING_DIR}/sdx_info


export SDX_NETWORK='sharednet1'
export SDX_NODE_NAME=${USERNAME}-sdx
export SDX_STACK_NAME=${USERNAME}-sdx





#RIAK Info
export RIAK_INFO_FILE=${WORKING_DIR}/riak_info
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`
echo RIAK_PUBLIC_IP: $RIAK_PUBLIC_IP

export OS_REGION_NAME=`cat $UNIVERSAL_INFO_FILE | grep OS_REGION_NAME | cut -d " " -f 2`
export OS_PROJECT_NAME=`cat $UNIVERSAL_INFO_FILE | grep OS_PROJECT_NAME | cut -d " " -f 2`
echo OS_REGION_NAME $OS_REGION_NAME
echo OS_PROJECT_NAME $OS_PROJECT_NAME



RIAK_PUBLIC_IP: 192.5.87.143
OS_REGION_NAME CHI@UC
OS_PROJECT_NAME CH-816532


In [20]:
#Clear and rebuild SDX info
echo "" > $SDX_INFO_FILE
echo SDX_NODE_NAME $SDX_NODE_NAME >> $SDX_INFO_FILE
echo SDX_STACK_NAME $SDX_STACK_NAME >> $SDX_INFO_FILE

#### Create Lease (if necessary)

In [21]:
#Check of universal reservation
if [ -f "${UNIVERSAL_INFO_FILE}" ]; then
   LEASE_NAME=`cat $UNIVERSAL_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
   echo $LEASE_NAME
   echo LEASE_NAME $LEASE_NAME >> $RIAK_INFO_FILE
fi

if  [ -z $LEASE_NAME ]; then
  
    echo No universal reservation. Creating new reservation

# --start-date "2019-02-10 17:50" \
# --end-date "2019-02-11 12:00" \
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`
LEASE_NAME=${USERNAME}"-sdx-lease"
echo LEASE_NAME $LEASE_NAME >> $SDX_INFO_FILE

NODE_MIN=1
NODE_MAX=1

NODE_TYPE='compute_haswell'
#NODE_TYPE='compute_skylake'

echo Creating Lease ${LEASE_NAME}
blazar lease-create \
   --physical-reservation min=${NODE_MIN},max=${NODE_MAX},resource_properties='["=", "$node_type", "'${NODE_TYPE}'"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${LEASE_NAME}

fi

echo LEASE_NAME: $LEASE_NAME
cat  $RIAK_INFO_FILE

pruth-cici-universal-lease2
LEASE_NAME: pruth-cici-universal-lease2

LEASE_NAME pruth-cici-universal-lease2
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc
RIAK_NODE_NAME pruth-riak
RIAK_STACK_NAME pruth-riak
RIAK_PUBLIC_IP 192.5.87.143
LEASE_NAME pruth-cici-universal-lease2
LEASE_NAME pruth-cici-universal-lease2
LEASE_NAME pruth-cici-universal-lease2
LEASE_NAME pruth-cici-universal-lease2


In [22]:
RESERVATION_ID=`blazar lease-show --format value -c id ${LEASE_NAME}`
echo RESERVATION_ID $RESERVATION_ID

RESERVATION_RESOURCE_ID=`blazar lease-show --format value -c reservations ${RESERVATION_ID} | grep \"id\" | cut -d "\"" -f 4`
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

#blazar lease-show ${RESERVATION_ID}

#Poll for node to be ready for ssh
while [[ true ]]
do 
	RESERVATION_STATUS=`blazar lease-show --format value -c status ${RESERVATION_ID}`
    
    if [[ "${RESERVATION_STATUS}" == "ACTIVE" ]]; then break; fi
    
    echo Reservation is ${RESERVATION_STATUS}. retrying in 5 seconds.
    sleep 5
done

echo RESERVATION_STATUS ${RESERVATION_STATUS}

echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID >> $SDX_INFO_FILE

cat $SDX_INFO_FILE


RESERVATION_ID 480a3fde-d50b-40d8-b48b-781729342b20
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc
RESERVATION_STATUS ACTIVE

SDX_NODE_NAME pruth-sdx
SDX_STACK_NAME pruth-sdx
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc


#### Start SDX Server

In [23]:
RESERVATION_RESOURCE_ID=`cat $SDX_INFO_FILE | grep RESERVATION_RESOURCE_ID | cut -d " " -f 2`
SDX_NODE_NAME=`cat $SDX_INFO_FILE | grep SDX_NODE_NAME | cut -d " " -f 2`
SDX_STACK_NAME=`cat $SDX_INFO_FILE | grep SDX_STACK_NAME | cut -d " " -f 2`

RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`


#IMAGE='CC-CentOS7'
IMAGE='CICI-SAFE.v1'

echo Creating SDX server. This will take several minutes!
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image $IMAGE \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${RESERVATION_RESOURCE_ID} \
                        --security-group default  \
                        --nic net-id=${SDX_NETWORK} \
                        --wait \
                        ${SDX_NODE_NAME}


#openstack stack create  --max-width 80 \
#        --template sdx.yaml \
#        --parameter "key_name=${SSH_KEY_NAME}" \
#        --parameter "reservation_id=${RESERVATION_RESOURCE_ID}" \
#        --parameter "network_name=${SDX_NETWORK}" \
#        --parameter "sdx_node_name=${SDX_NODE_NAME}" \
#        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
#        --wait \
#        ${SDX_STACK_NAME}

Creating SDX server. This will take several minutes!

+-------------------------------------+----------------------------------------+
| Field                               | Value                                  |
+-------------------------------------+----------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                 |
| OS-EXT-AZ:availability_zone         | nova                                   |
| OS-EXT-SRV-ATTR:host                | admin01-ironic                         |
| OS-EXT-SRV-ATTR:hypervisor_hostname | c03b5a7d-6b47-43d7-ab69-75ad943482d4   |
| OS-EXT-SRV-ATTR:instance_name       | instance-00008373                      |
| OS-EXT-STS:power_state              | Running                                |
| OS-EXT-STS:task_state               | None                                   |
| OS-EXT-STS:vm_state                 | active                                 |
| OS-SRV-USG:launched_at              | 2019-04-23T17:5

In [24]:
# Wait until server is ready before continuing
server_status=""
while [[ $server_status != "ACTIVE" ]]
do
   sleep 5
   echo Getting status for $SDX_NODE_NAME
   server_status=$(openstack server list --format value -c Status --name "$SDX_NODE_NAME")
   echo status $server_status
done

Getting status for pruth-sdx
status ACTIVE


In [25]:
# Request a public floating IP (in the 'public' network)
SDX_PUBLIC_IP=$(openstack floating ip create public --format value -c floating_ip_address)

echo SDX_PUBLIC_IP $SDX_PUBLIC_IP

# Assign a public floating IP to $USER-first-server
openstack server add floating ip "$SDX_NODE_NAME" "$SDX_PUBLIC_IP"
echo SDX_PUBLIC_IP $SDX_PUBLIC_IP >> $SDX_INFO_FILE

SDX_PUBLIC_IP 192.5.87.155


In [26]:
#SDX_NODE_NAME=`cat $SDX_INFO_FILE | grep SDX_NODE_NAME | cut -d " " -f 2`

#echo SDX server creation complete! 
#echo ${SDX_NODE_NAME} status `openstack server show  --format value -c status ${SDX_NODE_NAME}`


#SDX_PUBLIC_IP=`openstack server show  --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
#echo The SDX public IP is $SDX_PUBLIC_IP
#echo SDX_PUBLIC_IP $SDX_PUBLIC_IP >> $SDX_INFO_FILE


In [27]:
#SDX_PUBLIC_IP=`openstack server show --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
echo SDX_PUBLIC_IP is $SDX_PUBLIC_IP

#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo SDX node ready for ssh

SDX_PUBLIC_IP is 192.5.87.155
Node not ready for ssh. retrying in 5 seconds.
SDX node ready for ssh


In [28]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`

REMOTE_SCRIPT=/home/cc/script.sh
LOCAL_SCRIPT=${WORKING_DIR}/riak_script.sh

cat << EOF > $LOCAL_SCRIPT
#!/bin/bash
sudo sh -c 'echo "PermitRootLogin yes" >>/etc/ssh/sshd_config'
#sudo yum install -y docker vim mlocate git maven
sudo systemctl start docker

echo Start Plexus Controller in Container
#sudo docker pull yaoyj11/plexus
sudo docker run -i -t -d -p 8080:8080 -p 6633:6633 -p 3000:3000 -h plexus --name plexus yaoyj11/plexus
sudo docker exec -d plexus /bin/bash -c  "cd /root;pkill ryu-manager; ryu-manager ryu/ryu/app/rest_conf_switch.py ryu/ryu/app/rest_qos.py ryu/ryu/app/rest_router_mirror.py ryu/ryu/app/ofctl_rest.py"

echo Start SAFE Server in Container
#sudo docker pull yaoyj11/safeserver-v7
sudo docker run -i -t -d -p 7777:7777 -h safe --name safe yaoyj11/safeserver-v7
sudo docker exec -d safe /bin/bash -c  "cd /root/safe;sed -i 's/RIAKSERVER/$RIAK_PUBLIC_IP/g' safe-server/src/main/resources/application.conf;./prdn.sh"
           
echo Create ssh key
sudo sh -c ' ssh-keygen -t rsa -b 4096  -P "" -f "/root/.ssh/id_rsa"  -q'
sudo sh -c ' cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys2'
sudo sh -c ' chmod 600 ~/.ssh/authorized_keys2'

echo Boot Script Done!
EOF

echo Copying script to cc@$SDX_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_SCRIPT \
    cc@${SDX_PUBLIC_IP}:${REMOTE_SCRIPT}
echo Done

echo Running script
ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} "chmod +x "$REMOTE_SCRIPT" && "$REMOTE_SCRIPT

echo Done

Copying script to cc@192.5.87.155
Done
Running script
Start Plexus Controller in Container
2a7edadf68d24f47632314fc503aee74e0db41b81d94246fd00ce8e3f411673d
Start SAFE Server in Container
62f5b301c47d1a7b1b0d2b45ca6db978d187958ed2c6330209a7462f9ee96e42
Create ssh key
Boot Script Done!
Done


#### Configure the SDX node

In [29]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
echo SDX_PUBLIC_IP is $SDX_PUBLIC_IP

#Copy GENI pem file to node
REMOTE_GENI_CERT=/home/cc/geni.pem
GENI_CERT=${HOME}/work/geni-pruth.pem

echo Copying GENI cert to cc@$SDX_PUBLIC_IP:~/geni.pem
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    ${GENI_CERT} \
    cc@${SDX_PUBLIC_IP}:${REMOTE_GENI_CERT}
echo Done

SDX_PUBLIC_IP is 192.5.87.155
Copying GENI cert to cc@192.5.87.155:~/geni.pem
Done


In [30]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`

LOCAL_SDX_CONFIG=${WORKING_DIR}/SDX/sdx.conf

REMOTE_SDX_CONFIG=/home/cc/sdx.conf
REMOTE_WORKING_DIR=/root
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin

DATAPLANE_SLICE_NAME=${USERNAME}-sdx-dataplane

cat << EOF > $LOCAL_SDX_CONFIG
config {
safe=true
slicename="${DATAPLANE_SLICE_NAME}"
type="server"
sshkey="/root/.ssh/id_rsa"
safekey="sdx"
exogenipem="${REMOTE_GENI_CERT}"
exogenism="https://geni.renci.org:11443/orca/xmlrpc"
ipprefix="192.168.128.1/24"
riakserver="${RIAK_PUBLIC_IP}"
serverinslice=false
safeserver="${SDX_PUBLIC_IP}"
plexusserver="${SDX_PUBLIC_IP}"
serverurl="http://0.0.0.0:8888/"
scriptsdir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/scripts/"
clientsites:"UFL:UNF"
controllersite:"UFL (Gainesville, FL USA) XO Rack"
serversite:"RENCI (Chapel Hill, NC USA) XO Rack"
}
EOF

echo Copying SDX config to cc@$SDX_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_SDX_CONFIG \
    cc@${SDX_PUBLIC_IP}:${REMOTE_SDX_CONFIG}
echo Done

# apt-get install python-scapy
##!/usr/bin/env python
#import sys
#from scapy.all import *
#
#if __name__ == "__main__":
#    sendp(Ether(dst="ff:ff:ff:ff:ff:ff",src="12:34:56:78:9a:bc"), iface="ens6")

##!/usr/bin/env python
#import sys
#from scapy.all import *
#
#if __name__ == "__main__":
#        input = sniff(filter="ether src 12:34:56:78:9a:bc", count=1)
#        input[0].show2()


##!/usr/bin/env python
#import sys
#from scapy.all import *
#iface=str(sys.argv[1])
#dst=str(sys.argv[2])
#src=str(sys.argv[3])
#print 'iface = ', str(iface)
#print 'dst = ', str(dst)
#print 'src = ', str(src)
#sendp(Ether(dst=dst,src=src), iface=iface)
##sendp(Ether(dst="ff:ff:ff:ff:ff:ff",src="12:34:56:78:9a:bc"), iface="ens6")

##!/usr/bin/env python
#import sys
#from scapy.all import *
#src=str(sys.argv[1])
#print "src = ", str(src)
#filterStr="ether src "+str(src)
#input = sniff(filter=filterStr, count=1)
#input[0].show2()




Copying SDX config to cc@192.5.87.155
Done


#### Start the Dataplane Slice

In [31]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`

echo Start SDX dataplane Slice
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo ${REMOTE_BIN_DIR}/SafeSdxSliceServer -c $REMOTE_SDX_CONFIG  
echo Slice Creation Done!

Start SDX dataplane Slice
Slice Creation Done!


#### Start the SDX Server

In [ ]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`

echo Start the SDX Server
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    "sudo pkill java; sudo ${REMOTE_BIN_DIR}/SafeSdxServer -c $REMOTE_SDX_CONFIG &"

Start the SDX Server
starting exoplex.sdx server
18:08:56 [main] INFO  exoplex.sdx.core.SdxManager - Carrier Slice server with Service API: START
Sdx server has started, listening on http://0.0.0.0:8888/


# Please Cleanup Your Resources!!!

#### Delete Dataplane Slice

In [ ]:
#need to install exogeni phython script in the container
#python delete ${DATAPLANE_SLICE_NAME}

echo Delete SDX Slice

echo $REMOTE_SDX_CONFIG

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxSliceServer -c $REMOTE_SDX_CONFIG -d"
    
echo Slice Deletion Done

#### Clean Up SDX Chaeleon Stack and Reservation

In [ ]:
SDX_STACK_NAME=`cat $SDX_INFO_FILE | grep SDX_STACK_NAME | cut -d " " -f 2`
openstack stack delete --wait -y $SDX_STACK_NAME

LEASE_NAME=`cat $SDX_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
blazar lease-delete $LEASE_NAME

cat $SDX_INFO_FILE
mv $SDX_INFO_FILE ${SDX_INFO_FILE}.old